In [502]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from  itertools import chain

restaurant_df = pd.read_csv("/content/2020-XTern-DS.csv")
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 100)
pd.set_option('display.min_rows', 100)
pd.set_option('display.expand_frame_repr', True)

In [503]:
print(restaurant_df.dtypes)
print(len(restaurant_df))

Restaurant        object
Latitude         float64
Longitude        float64
Cuisines          object
Average_Cost      object
Minimum_Order     object
Rating            object
Votes             object
Reviews           object
Cook_Time         object
dtype: object
2019


In [504]:
restaurant_df['Rating'] = restaurant_df['Rating'].astype(str)
restaurant_df['Rating'] = restaurant_df['Rating'].str.replace(".","")

restaurant_df['Average_Cost'] = restaurant_df['Average_Cost'].str.replace("$","")
restaurant_df['Average_Cost'] = restaurant_df['Average_Cost'].str.replace(".","")
restaurant_df['Average_Cost'] = restaurant_df['Average_Cost'].apply(lambda x: int(x)/100 if x.isnumeric() else 0)

restaurant_df['Minimum_Order'] = restaurant_df['Minimum_Order'].str.replace("$","")
restaurant_df['Minimum_Order'] = restaurant_df['Minimum_Order'].str.replace(".","")
restaurant_df['Minimum_Order'] = restaurant_df['Minimum_Order'].apply(lambda x: int(x)/100 if x.isnumeric() else 0)

restaurant_df['Rating'] = restaurant_df['Rating'].apply(lambda x: int(x)/10 if x.isnumeric() else 0)
restaurant_df['Votes'] = restaurant_df['Votes'].apply(lambda x: int(x) if x.isnumeric() else 0)
restaurant_df['Reviews'] = restaurant_df['Reviews'].apply(lambda x: int(x) if x.isnumeric() else 0)
restaurant_df['Cook_Time'] = restaurant_df['Cook_Time'].apply(lambda x: int(x[:2]))
restaurant_df['Cuisines'] = restaurant_df['Cuisines'].apply(lambda x: [item.strip(' ') for item in x.split(',')])

# Pre 
The cleaning of this data set is extremely necessary since the dataset contains some restaurants that are in the state of opening or palnned only. \\
The cound of each restaurant style is calculated through correlated style tag with each restaurant and one restaurant could simultaneously represent multiple restaurant style tags, which makes sense since customers might go to one of the restaurant just because it provides one or two kinds of food. 

In [505]:
restaurant_effective_df = restaurant_df[restaurant_df['Rating'] != 0]
restaurant_effective_df['RV_Ratio'] = restaurant_effective_df['Reviews']/restaurant_effective_df['Votes']

restaurant_new_df = restaurant_df[restaurant_df['Rating'] == 0]

restaurant_effective_df.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Restaurant,Latitude,Longitude,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Cook_Time,RV_Ratio
0,ID_6321,39.262605,-85.837372,"[Fast Food, Rolls, Burger, Salad, Wraps]",20.0,50.0,3.5,12,4,30,0.333333
1,ID_2882,39.775933,-85.740581,"[Ice Cream, Desserts]",10.0,50.0,3.5,11,4,30,0.363636
2,ID_1595,39.253436,-85.123779,"[Italian, Street Food, Fast Food]",15.0,50.0,3.6,99,30,65,0.303030
3,ID_5929,39.029841,-85.332050,"[Mughlai, North Indian, Chinese]",25.0,99.0,3.7,176,95,30,0.539773
4,ID_6123,39.882284,-85.517407,"[Cafe, Beverages]",20.0,99.0,3.2,521,235,65,0.451056
5,ID_5221,39.370441,-85.739516,"[South Indian, North Indian, Chinese]",15.0,50.0,3.8,46,18,30,0.391304
6,ID_3777,39.821806,-85.005577,"[Beverages, Fast Food]",15.0,50.0,3.7,108,31,30,0.287037
7,ID_745,39.280324,-85.144363,"[Chinese, Thai, Asian]",65.0,50.0,4.0,1731,1235,45,0.713460
8,ID_2970,39.268816,-85.602168,"[Mithai, Street Food]",10.0,50.0,3.9,110,26,30,0.236364
9,ID_3474,39.874521,-85.439963,"[Fast Food, North Indian, Rolls, Chinese, Momo...",20.0,50.0,3.9,562,294,65,0.523132


In [506]:
restaurant_effective_df.sort_values(by='Average_Cost',ascending=False).head(20)

,Restaurant,Latitude,Longitude,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Cook_Time,RV_Ratio
2018,ID_8403,39.460944,-85.484355,"[Desserts, Beverages]",150.0,50.0,4.1,271,172,45,0.634686
225,ID_6278,39.225163,-85.958715,"[Chinese, Cantonese]",100.0,50.0,4.7,441,259,30,0.587302
1459,ID_3192,39.846970,-85.624772,"[North Indian, Mughlai, Kebab]",95.0,50.0,4.1,2340,1133,45,0.484188
784,ID_5879,39.072384,-85.734527,"[North Indian, Mughlai, Kebab]",90.0,450.0,4.2,1262,758,65,0.600634
683,ID_5879,39.416346,-85.061462,"[North Indian, Mughlai, Kebab]",90.0,450.0,4.2,1262,758,65,0.600634
1544,ID_1804,39.031950,-85.613019,"[European, Mediterranean]",85.0,50.0,4.1,1261,652,30,0.517050
973,ID_2421,39.351230,-85.012645,"[Continental, North Indian, Mughlai, Italian, ...",80.0,50.0,4.5,2831,1415,30,0.499823
1180,ID_1064,39.587777,-85.564805,"[Continental, North Indian, BBQ, Steak]",80.0,50.0,4.7,9054,6504,45,0.718357
520,ID_692,39.013650,-85.542853,"[Italian, North Indian, Chinese]",80.0,50.0,4.1,903,556,45,0.615725
510,ID_4276,39.932648,-85.976725,"[North Indian, Mughlai]",80.0,50.0,4.0,646,351,45,0.543344


#A.1
Conclusion 1 could be made through a simple checking on the total count of the current open restaurant in the area. It is obviously that indianapolis has most restaurants as "North Indian" style dining hall. The following one is Chinese restaurant while fast food is the third. The total number could represent a taste preference of the whole area. The choice of North Indian and Chinese is highly preferred and suuporte by the local residency. \\
Fast food chains could be so large because there are so many competitors on the market and their dining style is slightly harder to differentiate compared to traditional restaurant. \\
The restaurants could also represent the population consists statistics of indianapolis. +

In [507]:
cuisines_style_list = pd.Series([item for sublist in restaurant_effective_df['Cuisines'] for item in sublist])
cuisines_count_df = cuisines_style_list.value_counts().sort_index().rename_axis('Cuisines').reset_index(name='Count')
cuisines_count_df = cuisines_count_df.sort_values(by='Count', ascending=False, inplace=False)
cuisines_count_df.head(20)

,Cuisines,Count
62,North Indian,747
24,Chinese,537
29,Fast Food,454
75,South Indian,220
12,Beverages,208
27,Desserts,172
14,Biryani,152
59,Mughlai,148
79,Street Food,121
40,Italian,111


#A.2 
The second simple graph as we represented above is a sorted restaurants count for business that has a high rating (rating higher than 4.0). Even though we only showed top 10 count of each kind of restaurant. The rank difference between the first top 10 and the second 10 is noticeable. \\
Drastic decrease of tag "South Indian" is a sign of popular street food type with generally worse quality, while "North Indian" is obviously on the opposite. \\
Great increment in ranking for food tag such as "Italian" is a sign that these kind of restaurant normally will be more elegant trend that every restaurant will tend to make the business small but elite. \\
In order to make this clear, a ratio between high rated restaurants style count and total restaurants style count should be examined. \\

However, at this point, we could already be able to figure out that the North Indian, Chinese and Fast Food market are the most cmopetitve, but also most needed food in the area. Stepping into this business field means highly competitve business but larger potential customers.

In [508]:
restaurant_high_rating_df = restaurant_effective_df.sort_values(by='Rating', ascending=False)
restaurant_high_rating_df = restaurant_high_rating_df[restaurant_high_rating_df['Rating'] >= 4.0]
cuisines_high_style_list = pd.Series([item.strip(' ') for sublist in restaurant_high_rating_df['Cuisines'] for item in sublist])
cuisines_high_count_df = cuisines_high_style_list.value_counts().sort_index().rename_axis('Cuisines').reset_index(name='Count')
cuisines_high_count_df = cuisines_high_count_df.sort_values(by='Count', ascending=False, inplace=False)
cuisines_high_count_df.head(20)

,Cuisines,Count
50,North Indian,139
20,Chinese,95
25,Fast Food,83
10,Beverages,65
34,Italian,59
23,Desserts,59
22,Continental,49
17,Cafe,44
49,Mughlai,38
60,South Indian,32


#B
The chart below is a "High-end Count / Total Count" ratio for each kind of restaurant style. However, since there are too many cuisines style that only had appeared once that the ratio reached 100%, a cap of 10 count at least is being established. \\
In this chart, an intersting fact could be discussed is that most popular cuisines style such as "North Indian", "Chinese" and "Fast Food" could not be found at the top of the list. While on the opposite, all of them remain on the end of the list. \\
However, other cuisines style restaurants replace their place at the top of the list of high-end ratio. "Mexican" style of cuisine take a extreme high ratio of 0.7. This indicates that most of the Mexican style restaurants in Indianapolis is highly rated and most regarded as good quality food overall. Although the total appearance of Mexican style of cuisines is not high, but everyone is making a place in this market. \\
This is a clear sign that cuisines style such as "Mexican", "Asian", "Salad", "Continental" are maintain its overall reputation in this area. Joining in the business competition requires great amount of consideration and strating budget. Customers might focus more on the services for some of those tags while focus more on flavor and taste on other tags. Therefore, a more comprehensive research about the cuisine style to start is highly important. \\
Combined with discussion in part **A**, we could see that the most popoular food: "North Indian" and "Chinese" are highly popular and hold large market share while maintain at a low level of luxury, which gives a low starting point for every business man. It might be a good suggestion to start business with a relative stable customers group becuase the daily consumption is high enough to sustain the business.

In [509]:
cuisines_high_perc_df = cuisines_high_count_df[['Cuisines','Count']]
cuisines_high_perc_df['Ratio'] = cuisines_high_perc_df.apply(lambda x: x['Count'] / cuisines_count_df.loc[cuisines_count_df['Cuisines'] == x['Cuisines'], 'Count'].iloc[0], axis=1)
cuisines_high_perc_df[cuisines_high_perc_df['Count'] > 10].sort_values(by='Ratio',ascending=False)

,Cuisines,Count,Ratio
44,Mexican,11,0.733333
3,Asian,22,0.687500
57,Salad,27,0.586957
22,Continental,49,0.583333
0,American,11,0.578947
17,Cafe,44,0.556962
34,Italian,59,0.531532
67,Thai,16,0.484848
15,Burger,21,0.466667
38,Kebab,12,0.444444


#B.1
Simple effect could be seen through the newly opening restaurants in this area. A huge number of North Indian Restaurant, 131, are about to be openned and start to competing for the food market. Every food business with those cuisines style might have smaller scale in terms of restaurants size and price. This gives advantages in felxibilty in location while the tiny scale gives disadvantage in growing large and maintain popular reputation.

In [510]:
cuisines_new_style_list = pd.Series([item.strip(' ') for sublist in restaurant_new_df['Cuisines'] for item in sublist])
cuisines_new_count_df = cuisines_new_style_list.value_counts().sort_index().rename_axis('Cuisines').reset_index(name='Count')
cuisines_new_count_df = cuisines_new_count_df.sort_values(by='Count', ascending=False, inplace=False)
cuisines_new_count_df.head(10)

,Cuisines,Count
35,North Indian,131
18,Fast Food,107
13,Chinese,94
7,Beverages,55
9,Biryani,33
16,Desserts,32
42,South Indian,28
33,Mughlai,22
39,Rolls,20
43,Street Food,20


#C
After go through the ranking of each cuisine style, we could start doing something different by count number but with simple rating across cuisine style. \\
The code below is a simple examination of the highest rated cuisines style across the board. The result is similar to what we have in Part **B**. However, it still differentiates in mathmatical sense. The Mean rating, according to its attributes, could easily be affected by extreme value, such as outlier in its rating. \\
From the chart below we could see that restaurant that has a high maintained good rating among all of its own kind also tends to have high average rating. This could also be seen as this kind of cuisines styles will also has lower variance in their rating distribution, which gives them the edge of maintaining high rating across the board.

In [511]:
cuisines_mean_df = restaurant_effective_df[['Cuisines','Rating']]
cuisines_count_df['Mean'] = 0.0

for i in range(len(cuisines_count_df)):
  cuisines_mean_df['Contain'] = restaurant_effective_df['Cuisines'].apply(lambda x: cuisines_count_df['Cuisines'][i] in x)
  the_mean = cuisines_mean_df.groupby('Contain').mean()
  cuisines_count_df['Mean'][i] = float(the_mean.take([True])['Rating'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [512]:
cuisines_effective_df = cuisines_count_df[cuisines_count_df['Count'] > 10]
cuisines_effective_df.sort_values(by='Mean',ascending=False).head(20)

,Cuisines,Count,Mean
28,European,12,4.166667
54,Mexican,15,4.160000
4,Asian,32,4.034375
30,Finger Food,14,4.021429
72,Salad,46,3.976087
1,American,19,3.968421
26,Continental,84,3.961905
40,Italian,111,3.929730
20,Cafe,79,3.905063
44,Kebab,27,3.900000


#D
In a more mathmatical approach, we will try to quantify the possible ratins of a restaurant based on values such as average costs and minimun order, out side of cuisines style. In the code below, we use two simple linear regression model: the pure linear regression and ridge regression. In our implementation, we tried to penalize it to regularized the regression.\\

In [513]:
import sklearn
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split

In [514]:
def L2_error(y, y_hat):
  return np.mean((y - y_hat) ** 2)

def L1_error(y, y_hat):
  return np.mean(np.absolute(y - y_hat))


def LinearRegression(train_data, train_labels):
  tail = np.full((train_data.shape[0], 1), 1)
  X = np.hstack((train_data, tail))
  w_prime = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(train_labels)
  # print(w_prime)
  w = w_prime[:-1]
  # print(w)
  b = w_prime[-1][0]
  # print(b)
    
    
  return (w,b)

def RidgeRegression(train_data, train_labels, lambda_):
  tail = np.full((train_data.shape[0], 1), 1)
  X = np.hstack((train_data, tail))
  w_prime = np.linalg.inv(np.add(X.T.dot(X), np.identity(X.shape[1]) * lambda_)).dot(X.T).dot(train_labels)
  w = w_prime[:-1]
  b = w_prime[-1][0]
    
  return (w,b)

In [515]:
regression_df = restaurant_effective_df[['Average_Cost','Minimum_Order','Cook_Time', 'Votes', 'Reviews', 'RV_Ratio', 'Rating']]
regression_df = regression_df.dropna(axis=0)
regression_df = regression_df[(regression_df != 0).all(1)]
X = regression_df[['Average_Cost','Minimum_Order','Cook_Time', 'Votes', 'Reviews']]
y = regression_df['Rating'].to_numpy().reshape(len(regression_df['Rating']), 1)
transformer = Normalizer().fit(X)
X = transformer.transform(X)
X = np.hstack((X, regression_df['RV_Ratio'].to_numpy().reshape(len(regression_df['RV_Ratio']), 1)))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

#D.1
In our final result, we could see that using linear regression will gives us the highest weight at Reviews. This is a obvious from our intuition that restaurants with higher rating often tends to be popular and attract more customers which gives a higher potential reviews writer group. \\

In our second attempt using Ridge regression, we could see that the highest weight is, instead, being trained on number of votes. This also agrees with our intuition that more votes means more customers are impressed this specific restaurant. However, the different highest weight coming from two different regression model could be that linear regression have no regularization that the extreme value of number of "Reviews" is affected the weight of each factor. \\

In a conclusion term, the average-cost, minimum-order, cooking time, are not great factors to be taken into consideration while evaluating a potential profit ability of a new food business. other factors, such as services, food quality, convenience, the factors that are slightly little harder to quantify might affect more in making a restaurant more welcome and popular. \\

While the prediction error on two model do not differentiate greatly, this means the most important factors influencing a restaurant's rating is a higher number of votes and reviews. This leaves the question that, do those restaurants get a lot of votes and reviews because they have high rating, or do those restaurants get a high rating because they have a lot of votes and reviews (the later one obviously wrong in intuition, but what about an influencer restaurant or Matthew effect?)

In [516]:
w, b = LinearRegression(X_train, y_train)
z_train = X_train.dot(w) + b
z_test = X_test.dot(w) + b
print(L1_error(y_train, z_train))
print(L2_error(y_train, z_train))
print(L1_error(y_test, z_test))
print(L2_error(y_test, z_test))
print(w)

0.20602449583930915
0.07734770639114692
0.2056241632652607
0.0765198655733908
[[-0.36383409]
 [ 0.08996119]
 [-0.60435654]
 [ 0.11500351]
 [ 1.19685557]
 [-0.14690347]]


In [517]:
for l in [5, 10, 20, 50]:
  w, b = RidgeRegression(X_train, y_train, l)
  z_train = X_train.dot(w) + b
  z_test = X_test.dot(w) + b
  print("Lambda: ", l)
  print(L1_error(y_train, z_train))
  print(L2_error(y_train, z_train))
  print(L1_error(y_test, z_test))
  print(L2_error(y_test, z_test))
  print(w)
  print()

Lambda:  5
0.21634969550522545
0.08553330699845714
0.21854712419830416
0.08471849116709741
[[-0.13405795]
 [ 0.42372324]
 [-0.13153985]
 [ 0.9525256 ]
 [ 0.67406197]
 [ 0.3461735 ]]

Lambda:  10
0.225319739735213
0.09099672682887612
0.22860328865842344
0.08986869743637423
[[-0.03441755]
 [ 0.49316726]
 [ 0.04128722]
 [ 1.14620018]
 [ 0.60775733]
 [ 0.45745109]]

Lambda:  20
0.236044295763785
0.09728928877698881
0.24069208648150933
0.09630273793916154
[[0.05524158]
 [0.53863483]
 [0.18631761]
 [1.26557238]
 [0.57526038]
 [0.55039944]]

Lambda:  50
0.2584789343087079
0.11072687568004276
0.26669028986594007
0.11229033657706829
[[0.13980805]
 [0.57926863]
 [0.3139465 ]
 [1.30251447]
 [0.55603679]
 [0.64665297]]

